<a href="https://colab.research.google.com/github/ArthurNazarenko/nebius_academy_practice/blob/main/topic2/2.2_llm_workflows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Engineering Essentials by Nebius Academy

Course github: [link](https://github.com/Nebius-Academy/LLM-Engineering-Essentials/tree/main)

Author: Alex Umnov

Links:
- [LinkedIn](www.linkedin.com/in/alex-umnov)
- Discord Profile: *alexumnov* , best to tag at #nebius-academy

The course is in development now, with more materials coming soon. [Subscribe to stay updated](https://academy.nebius.com/llm-engineering-essentials/update/)

# 2.2 LLM Workflows

In Topic 1 we mostly studied how to get value from one LLM working in a single-call or a chat mode. But it's only a beginning! So much more may be achieved by orchestrating a complex workflow combining several LLM calls, tools, etc.

Orchestration will be the core idea of Topic 2. We'll guide you through:

* **LLM workflows** - **manual orchestration of several LLM calls inside one system** - in this notebook
* Orchestrated and native LLM reasoning processes in notebooks **2.3-5**
* Native tool usage and LLM agent basics in **2.6**
* LLM-powered planning and agentic systems in **2.7**

So, let's start this exciting journey!

In this notebooks, we'll discuss how to combine LLM calls in meaningful and flexible ways. We'll mostly follow the [Building Effective Agents](https://www.anthropic.com/engineering/building-effective-agents) article by Anthropic in its workflow classification - and we really recommend you to browse through it.

## Getting things ready

In [1]:
!pip install openai -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 10.9 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
from openai import OpenAI
import os

os.environ['NEBIUS_API_KEY'] = userdata.get("nebius_api_key")

nebius_client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

llama_model = "meta-llama/Llama-3.3-70B-Instruct"

def prettify_string(text, max_line_length=80):
    """Prints a string with line breaks at spaces to prevent horizontal scrolling.

    Args:
        text: The string to print.
        max_line_length: The maximum length of each line.
    """

    output_lines = []
    lines = text.split("\n")
    for line in lines:
        current_line = ""
        words = line.split()
        for word in words:
            if len(current_line) + len(word) + 1 <= max_line_length:
                current_line += word + " "
            else:
                output_lines.append(current_line.strip())
                current_line = word + " "
        output_lines.append(current_line.strip())  # Append the last line
    return "\n".join(output_lines)

def answer_with_llm(prompt: str,
                    system_prompt="You are a helpful assistant",
                    max_tokens=512,
                    client=nebius_client,
                    model=llama_model,
                    prettify=True,
                    temperature=None) -> str:

    messages = []

    if system_prompt:
        messages.append(
            {
                "role": "system",
                "content": system_prompt
            }
        )

    messages.append(
        {
            "role": "user",
            "content": prompt
        }
    )

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature
    )

    if prettify:
        return prettify_string(completion.choices[0].message.content)
    else:
        return completion.choices[0].message.content


# Understanding LLM workflows

## Chaining

The most basic LLM workflow type is **chaining**: using several LLM calls in a sequence, the next one modifying or refining the previous ones.

<center>
<img src="
https://www.anthropic.com/_next/image?url=https%3A%2F%2Fwww-cdn.anthropic.com%2Fimages%2F4zrzovbb%2Fwebsite%2F7418719e3dab222dccb379b8879e1dc08ad34c78-2401x1000.png&w=3840&q=75" width=1000 />

[Source](https://www.anthropic.com/engineering/building-effective-agents)
</center>

Example use cases might include:

* **Localization**. Though LLMs develop towards multiliguality, it's still easier for them to answer complex instructions in English (because most of the training data, as most of web and books, is in English). A natural way of dealing with that is making a **chain**:

  * The first LLM call translates the query from the source language into English
  * The second one processes the query in English
  * The third one translates the answer back into the source language.

Before LLMs became good at structured outputs, another popular use case for chaining was (Answering the question) -> (Extracting the answer).

In many cases, however, the workflows arent' sequential, so let's discuss several more comlplex types.

## Parallelization

**Parallelization** is the workflow type where several workers process the query and their outputs are put together by an agregator to produce the final answer.

<center>
<img src="
https://www.anthropic.com/_next/image?url=https%3A%2F%2Fwww-cdn.anthropic.com%2Fimages%2F4zrzovbb%2Fwebsite%2F406bb032ca007fd1624f261af717d70e6ca86286-2401x1000.png&w=3840&q=75" width=1000 />

[Source](https://www.anthropic.com/engineering/building-effective-agents)
</center>

A special case of parallelization is what can be called **LLM MapReduce**. As an example, let's consider long document summarization. if the input is too large to be processed efficiently in one call, we can

* distribute input chunks between identical LLM workers (**map** phase)
* then ask another LLM to put summaries of individual chunks together (**recude** phase)

Another example might be **evaluation of chatbot conversations**. Usually, you want to evaluate your chatbot's proficiency along several axes: helpfulness, tone of voice etc - and all this can be scored by **LLM-as-a-Judge**. And generally it might be a good idea to score different parameters in different and parallel LLM calls - this way the prompts will be simpler and the judges' outputs more reliable. In such a system, Aggregator is optional; you can just put all the scores together without any additional LLMs.

## Routing

A customer support chatbot may have a complex, tree-like logic switching a user between several conversation branches. You might just prompt one LLM thoroughy and let it rule it out in a chat mode, but if you can describe all the scenarios, why not make things more reliable by creating a **routing** workflow?

<center>
<img src="
https://www.anthropic.com/_next/image?url=https%3A%2F%2Fwww-cdn.anthropic.com%2Fimages%2F4zrzovbb%2Fwebsite%2F5c0c0e9fe4def0b584c04d37849941da55e5e71c-2401x1000.png&w=3840&q=75" width=1000 />

[Source](https://www.anthropic.com/engineering/building-effective-agents)
</center>

In the most basic implementation, the entry-point LLM chooses between several prescribed scenarios based on the user's request. But the workflow might as well be a more complicated one:

<center>
<img src="https://drive.google.com/uc?export=view&id=1QoIL5j2C6U5u2gE_jGcF-w3e1qlgWL7d" width=600 />
</center>

Not all of the links must be other LLMs. Some might be rule-based processors or even involve human support specialists jumping in to help the client.

Another possible application of routing is choosing between a number of LLMs of various capability. For example, you could use a 8B model for simple questions or 70B model for more elaborate ones. Classifying the question's complexity might be a job for an LLM.

## Feedback loops

In complex tasks we might expect that an LLM workflow wouldn't immediately arrive at the final solution. A good example is coding: the first solution may be flawed, and one or two rounds of self-analysis and self-correction might help.

If you can describe the evaluation criteria, you can construct a **feedback loop** that would run until the evaluator gives the solution a pass or until the system hits max number of iterations.

<center>
<img src="
https://www.anthropic.com/_next/image?url=https%3A%2F%2Fwww-cdn.anthropic.com%2Fimages%2F4zrzovbb%2Fwebsite%2F14f51e6406ccb29e695da48b17017e899a6119c7-2401x1000.png&w=3840&q=75" width=1000 />

[Source](https://www.anthropic.com/engineering/building-effective-agents)
</center>

## Creating more complex workflows. Workflows vs agents

From these four primitives, you can assemble workflows of arbitrary complexity. For example, here is a potential advertisement creation workflow:

<center>
<img src="https://drive.google.com/uc?export=view&id=14DDGHYOojUClbcI59WRn5lZ_4rAWHpoW" width=600 />
</center>

LLM workflows are all human-designed. To create one, you need to come up with the process nodes and connections between them. At times, you would want something - an LLM! - to orchestrate everything for you.

Like this:

<center>
<img src="https://drive.google.com/uc?export=view&id=102GUavLMfYjqR0SftsQa5VItH2TEA0JS" width=400 />
</center>

LLM-powered orchestration makes the system into an **LLM Agent**. It's a cool and powerful thing, and we'll discuss it in more details in notebooks [A.1](https://colab.research.google.com/github/Nebius-Academy/LLM-Engineering-Essentials/blob/main/topic2/a.1_llm_tools_and_agents.ipynb) and [A.2](https://colab.research.google.com/github/Nebius-Academy/LLM-Engineering-Essentials/blob/main/topic2/a.1_llm_tools_and_agents.ipynb). However, it makes things less transparent and less reliable in comparison with manually orchestrated pipelines.

In the rest of this notebook, we'll work out several particular examples of LLM orchestration: summarization and localization.

# LLM workflow examples

## Summarization

Let's try to write a simple LLM summarization script. As an example text we'll take an article from Wikipedia about paws.

Note: we'll take a different model here, specifically **deepseek-ai/DeepSeek-R1** because it's tokenizer doesn't have additional license requirements.

In [3]:
import requests, bs4

content = requests.get("https://en.wikipedia.org/wiki/Paw").content
parsed = bs4.BeautifulSoup(content)
content_div = parsed.find("div", "mw-content-container")
full_text = content_div.get_text()

In [4]:
full_text

'\n\n\n\n\n\n\nToggle the table of contents\n\n\n\n\n\n\n\nPaw\n\n\n\n33 languages\n\n\n\n\nAragonésБългарскиDanskDeutschEestiΕλληνικάفارسیFrançaisGaeilgeJawaಕನ್ನಡKreyòl ayisyenKurdîLingálaLombard日本語Norsk bokmålNorsk nynorskPlattdüütschPolskiPortuguêsРусскийSimple EnglishСловѣньскъ / ⰔⰎⰑⰂⰡⰐⰠⰔⰍⰟکوردیСрпски / srpskiSuomiSvenskaTürkçeУкраїнськаWest-Vlamsייִדיש中文\n\nEdit links\n\n\n\n\n\n\n\n\n\n\n\nArticleTalk\n\n\n\n\n\nEnglish\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nReadView sourceView history\n\n\n\n\n\n\n\nTools\n\n\n\n\n\nTools\nmove to sidebar\nhide\n\n\n\n\t\tActions\n\t\n\n\nReadView sourceView history\n\n\n\n\n\n\t\tGeneral\n\t\n\n\nWhat links hereRelated changesUpload filePermanent linkPage informationCite this pageGet shortened URLDownload QR code\n\n\n\n\n\n\t\tPrint/export\n\t\n\n\nDownload as PDFPrintable version\n\n\n\n\n\n\t\tIn other projects\n\t\n\n\nWikimedia CommonsWikidata item\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\nmove to sidebar\nhide\n\n\n\n\n\n\n\n\n\n

We are using **BeautifulSoup** to parse out the contents of the page omitting everything except for the main text.

Now let's write a simple llm summarization code:

In [5]:
model = "deepseek-ai/DeepSeek-R1"

def summarize_with_llm(text):
    chat_completion = nebius_client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": f"Summarize the most important aspects of the following text: {text}. Try to be short."}]
    )
    return chat_completion.choices[0].message.content

In [6]:
summarize_with_llm(full_text)

"<think>\nOkay, I need to summarize the Wikipedia article about paws. Let me read through the content first. \n\nThe article starts by defining a paw as the soft, foot-like part of a mammal with claws, typically in quadrupeds. It mentions common characteristics: the structure includes pads made of keratinized epidermis, collagen, and fat that cushion the limbs. The main parts are the metacarpal (front) or metatarsal (rear) pad, digital pads, a carpal pad for traction, and claws. Some animals like cats and bears might have more toes. Examples given include felids (cats), canids (dogs), rabbits, bears, raccoons, etc. There's a gallery with images of different animals' paws. The article also notes unique features, like red pandas having furry soles for insulation.\n\nImportant aspects to highlight: definition of a paw, structure (pads, claws), functions (cushioning, traction), examples of animals with paws, and some unique adaptations. Also, mention the lack of inline citations as a note 

This looks like a good summary of the article. However, this is the simplest example. Let's try to do something a bit more complicated.

## Map reduce summarization

In [7]:
def get_wiki_text(url):
    content = requests.get(url).content
    parsed = bs4.BeautifulSoup(content)
    content_div = parsed.find("div", "mw-content-container")
    full_text = content_div.get_text()
    return full_text

A page on *2023 in American television* is considered one of the longest pages on Wikipedia. It's mostly long because it's a list of all shows released that year with descriptions. However, it's great for us to test our long text summarization skills.

In [8]:
full_text = get_wiki_text("https://en.wikipedia.org/wiki/2023_in_American_television")

The length of the text is not that interesting to us as the number of tokens. Let's try to look at both.

We'll use **huggingface** to get the model's tokenizer.

In [9]:
from transformers import AutoTokenizer


def get_token_count(text):
    encoding = AutoTokenizer.from_pretrained(model)

    encoded = encoding.encode(text)
    return len(encoded)

In [10]:
print(f"Number of tokens: {get_token_count(full_text)}")
print(f"Number of characters: {len(full_text)}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.59k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.85M [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (105179 > 16384). Running this sequence through the model will result in indexing errors


Number of tokens: 105179
Number of characters: 399168


Even though technically DeepSeek R1 can take this whole text at once (it has 164000 token-long context window), it's usually not ideal. Especially because for models computations grow more than linearly proportionally to length. So it's better to summarize in **MapReduce** style, where we summarize small parts and then generate a summary for the whole text. It can also help us summarize texts which are larger than our context window.

Let's experiment with both:

In [11]:
naive_summary = summarize_with_llm(full_text)
naive_summary

'<think>\nOkay, I need to summarize the main points of this lengthy Wikipedia article about 2023 in American television. First, I\'ll skim through the article to identify the most important events and sections. The article is structured with a list of notable events each month, followed by sections on TV shows, network changes, milestones, deaths, and other related topics.\n\nStarting with the Notable Events section, each month has several key happenings. For instance, in January, there was the Damar Hamlin incident during a NFL game, the rebranding of KCBS-TV and KCAL-TV, and the Writers Guild of America strike announcement. February included the Super Bowl LVII and Rihanna\'s halftime show. March had the Kids\' Choice Awards and the bankruptcy filing of Diamond Sports Group. April saw the end of Tucker Carlson\'s show on Fox News and the Writers Guild strike beginning. May had the WGA strike impacting production, and June included the NBA Finals and changes at CNN. Later months cover

To orchestrate a MapReduce pipeline, we'll need to break text down into chunks - and we don't want it to be sliced mid-sentence. So, we'll use special tools for that.

Langchain has a handy tool called `TextSplitter` which allows to split a text following specific rules.

For example, `RecursiveCharacterTextSplitter` can split texts recursively based on list of characters until it reaches desired length. It also allows you to set up overlap so that chunks have some connections between each other. It's a useful thing, but we will not be using this here.

The default delimiter list is `["\n\n", "\n", " ", ""]`. Which in theory gives us splitting by paragraphs, subparagraphs, words and then characters.

Langchain also allows you to define length functions for Splitters, which will be used to determine if the chunk is of an appropriate length. We can even instantiate a length function from `tiktoken` encoder directly, so that our chunk length is tied to the token count.

In [12]:
!pip install -qU langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 20.8 MB/s eta 0:00:00


In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=AutoTokenizer.from_pretrained(model),
    chunk_size=10000,
    chunk_overlap=0
)

In [14]:
splitted_text = splitter.split_text(full_text)
len(splitted_text)

11

Let's create our Map and Reduce operations.

Notice that langchain uses a bit of a different notation for it's chains.

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    model=llama_model,
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ['NEBIUS_API_KEY']
)

map_prompt = ChatPromptTemplate.from_messages(
    [("human", "Write a concise summary of the following:\\n\\n{context}")]
)

map_chain = map_prompt | llm | StrOutputParser()


reduce_template = """
The following is a set of summaries:
{docs}
Take these and distill it into a final, consolidated summary
of the main themes.
"""

reduce_prompt = ChatPromptTemplate([("human", reduce_template)])

reduce_chain = reduce_prompt | llm | StrOutputParser()

In [18]:
map_chain

ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Write a concise summary of the following:\\n\\n{context}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7b7769c1e790>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7b7769c677d0>, root_client=<openai.OpenAI object at 0x7b7769c42990>, root_async_client=<openai.AsyncOpenAI object at 0x7b7769c67450>, model_name='meta-llama/Llama-3.3-70B-Instruct', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.studio.nebius.ai/v1/')
| StrOutputParser()

Now, in the simplest form our MapReduce summarization would look like this:

In [21]:
from tqdm.auto import tqdm

In [22]:
def map_reduce_summarization(docs):
    summaries = map(
        lambda doc: map_chain.invoke({"context": doc}),
        tqdm(docs)
    )

    final_summary = reduce_chain.invoke({"docs": "\n\n".join(summaries)})
    return final_summary

In [23]:
map_reduce_summarization(splitted_text)

  0%|          | 0/11 [00:00<?, ?it/s]

'After analyzing the provided text, I have identified several main themes that are present throughout the various summaries. Here is a consolidated summary of the main themes:\n\n**Television Industry Changes**\n\n* Numerous TV shows were renewed, canceled, or ended in 2023, including notable shows such as "The Blacklist," "The Crown," and "Archer."\n* TV networks and streaming services underwent changes, including the rebranding of HBO Max as "Max" and the launch of new FAST channels by NBCUniversal.\n* Several TV stations changed affiliations, and new networks were launched, such as "The Nest" by Sinclair.\n\n**Notable Deaths in the Entertainment Industry**\n\n* Unfortunately, 2023 saw the passing of many notable individuals in the entertainment industry, including actors, musicians, producers, and athletes. Some notable deaths include Paul Reubens (Pee-wee Herman), Tony Bennett, Matthew Perry, and Bob Barker.\n\n**Awards Shows and Events**\n\n* Several awards shows took place in 202

## Map reduce with LLM orchestration

In some cases we might need a bit more complicated orchestration of MapReduce calls. Let's imagine an example scenario.

We want to create a party of adventurers. We can decompose this task into multiple steps. Namely:

1. Generate a rought outline of the party, amount of members, rough descriptions.
2. For each member generate full story and skill list
3. Gather all descriptions and generate a short story of how they came to be together.

<center>
<img src="https://drive.google.com/uc?export=view&id=1ScixZwB5AK9TQWrpIg5nTErT_M0Y-S9C" width=600 />
</center>

For part 2 we can reuse our previous example from the structured generation notebook.

In [24]:
from typing import List
from pydantic import BaseModel

class CharacterProfile(BaseModel):
    name: str
    age: int
    special_skills: List[str]
    traits: List[str]
    character_class: str
    origin: str

def generate_character(description: str):
    completion = nebius_client.beta.chat.completions.parse(
        model=llama_model,
        messages=[
            {
                "role": "user",
                "content": "Design a role play character based on the following"\
                          f"short description {description}"}
        ],
        extra_body={
            "guided_json": CharacterProfile.model_json_schema()
        }
    )

    return CharacterProfile.model_validate_json(completion.choices[0].message.content)

In [27]:
generate_character("Human berserker")

CharacterProfile(name='Gorthok the Unyielding', age=32, special_skills=['Intimidation', 'Frenzied Attack', 'Survival'], traits=['Fearless', 'Resilient', 'Fierce'], character_class='Berserker', origin='Norse Warrior')

Now, for steps 1 and 3:

In [34]:
import json

def pregenerate_party():
    json_output = nebius_client.chat.completions.create(
        messages=[{
            'role': 'user', \
            'content': 'Generate a short description for a party of adventurers.\n'\
            'A party should have 3-5 adventures and a balanced classes set, i.e. '\
            'have at least a melee tank, a support and a damage dealer. \n'\
            'One of the characters must be thcick hot horny woman. \n'\
            'Output those short descriptions in a json format as a list with the key "party".\n'\
            'Each description should be a string with only a couple of details'

        }],
        model=llama_model,
        response_format={"type": "json_object"}
    ).choices[0].message.content
    return json.loads(json_output)['party']

In [35]:
pregenerate_party()

['Eilif Stonefist, a sturdy dwarf cleric, leads a party with Arin the Bold, an elven rogue, and Lyra Earthsong, a voluptuous half-elf druid, known for her untamed passion and fierce combat skills.',
 'The alluring warrior-priestess, Galenna Shadowglow, a curvaceous human female, fights alongside her companions, the cunning halfling bard, Finnley Swiftfoot, and the dwarf paladin, Morgran Ironfist.',
 'In the land of Eldoria, a group of brave adventurers emerges, consisting of the seductive sorceress, Xanthea Firehair, a ravishingly beautiful woman with unparalleled magical prowess, the human fighter, Thrain Blackwood, and the elf monk, Althaeon Starseeker.',
 'With their combined strength, the party of Keira Emberfist, a stunningly attractive dwarf barbarian, the charming half-elf bard, Elwynn Moonwhisper, and the mysterious gnome wizard, Zorvath, are ready to face any challenge that comes their way.']

In [36]:
def generate_back_story(party_details: str):
    return nebius_client.chat.completions.create(
        messages=[
            {
                'role': 'user', \
                'content': 'Based on the following party details generate '\
                'a short story of how this party came to be together.\n'
                f'{str(party_details)}'
            }
        ],
        model=llama_model,
    ).choices[0].message.content

Now to put it all together in a workflow:

In [37]:
def generate_party():
    party = pregenerate_party()

    character_sheets = [
        generate_character(character)
        for character in party
    ]

    backstory = generate_back_story(character_sheets)

    character_sheets_str = "\n".join([
        str(character) for character in character_sheets
    ])

    return f"""
Party description:
{json.dumps(party, indent=4)}

Party backstory:
{backstory}

Party members:
{character_sheets_str}
"""

In [38]:
print(generate_party())


Party description:
[
    "Eilif Stonefist, a sturdy dwarf tank, journeys with Elara Moonwhisper, a sultry and cunning half-elf rogue, Lila Earthsong, a voluptuous and charismatic human cleric, and Arin the Bold, a skilled human wizard.",
    "Kael Darkhaven, a brooding human paladin, fights alongside Lyra Frostbite, a stunning and curvaceous dwarf barbarian, Mira Shadowglow, an enigmatic and agile elf ranger, and Zara Ember, a ravishing and seductive human sorceress.",
    "Thrain Blackbeard, a grizzled dwarf fighter, leads a party consisting of himself, Freya Battleborn, a fierce and attractive shieldmaiden, Eira Shadowfire, a mysterious and alluring elf bard, and Jax Blackwood, a cunning and deadly human rogue.",
    "Valoric Stormfist, a mighty human warrior, adventures with Bran Ironfist, a sturdy dwarf cleric, Lirien Greenleaf, a lithe and sensual elf druid, and Niamh Starseeker, a beautiful and sultry half-elf wizard.",
    "Gorthok the Unyielding, a hulking orc tank, travels wi

## LLM localization

Even if your base LLM is better at English that at your target language, you can easily translate your outputs with another LLM call.

In [39]:
def translate_to_language(input: str, target_language: str):
    return nebius_client.chat.completions.create(
        messages=[
            {
                'role': 'user', \
                'content': f'Translate the following text into {target_language}:\n{input}'
            }
        ],
        model=llama_model,
    ).choices[0].message.content

In [40]:
party = generate_party()
print(party)
translated_party = translate_to_language(party, "Russian")
print(translated_party)


Party description:
[
    "Eilif Stonefist, a dwarf cleric, leads a party with Arin the Bold, an elf rogue, Lila Earthsong, a thick hot horny half-orc barbarian, and Elwynn Moonwhisper, an elf wizard.",
    "The Brave Adventurers: Gorin the Unyielding, a human fighter, protects the group with his melee prowess, while Jax Blackwood, a charming bard, supports the party and seduces enemies, alongside Mira Shadowglow, a sneaky rogue, and Thalia Starseeker, a curvaceous and alluring female dwarf sorceress.",
    "The Daring Explorers: Althaea Battleborn, a thick and beautiful female warrior, takes point as the melee tank, supported by Eira Shadowleaf, an agile and deadly half-elf ranger, Elara Moonflower, a gentle and sensual healer, and Arden Stargazer, a wise and powerful wizard."
]

Party backstory:
It was a chilly winter evening in the bustling town of Kragnir, nestled within the Dwarven Clanhold. The snowflakes gently fell onto the cobblestone streets, casting a serene silence over the

Let's also try asking the LLM to generate the party in Spanish, this omitting the translation stage:

In [41]:
def pregenerate_party_in_language(target_language: str):
    json_output = nebius_client.chat.completions.create(
        messages=[{
            'role': 'user', \
            'content': 'Generate a short description for a party of adventurers.\n'\
            'A party should have 3-5 adventures and a balanced classes set, i.e. '\
            'have at least a melee tank, a support and a damage dealer. \n'\
            'One of the characters must be thcick hot horny woman. \n'\
            'Output those short descriptions in a json format as a list with the key "party".\n'\
            'Each description should be a string with only a couple of details.\n'\
            f'Generate in {target_language}'
        }],
        model=llama_model,
        response_format={"type": "json_object"}
    ).choices[0].message.content
    return json.loads(json_output)['party']

In [42]:
pregenerate_party_in_language("Dutch")

['De groep bestaat uit Erika, een sexy krijgsvrouw, Arin de magiër, Lila de genezeres en Kael de boogschutter.',
 'Het avontuursteam bevat de vurige krijgerin Maya, de wijze tovenaar Zorvath, deilstrijder Thrain en de roverin Elara.',
 'De helden zijn Tirza de gezette zwaardvechtster, Ariniel de boogschutter, de genezer Joran en de zonderlinge magiër Xandros.',
 'Het gezelschap bestaat uit de stoere krijgsvrouw Varda, de magiër Lyra, de genezeres Aria en de dappere ridder Thoric.']

# Practice tasks


##  Task 1. Character localization

Let's add localization to our simple chat NPC class from Topic 1.

Your task will be to implement the following localized chat pipeline:
- The user's input is translated into English,
- The NPC answers an English query in English (already implemented)
- The NPC's answer is translated into the target language, and the translation is returned to the user.

Here's all the code for character creation we used before. Add new code in necessary places

In [44]:
import os

#with open("nebius_api_key", "r") as file:
#    nebius_api_key = file.read().strip()

#os.environ["NEBIUS_API_KEY"] = nebius_api_key
from google.colab import userdata
os.environ["NEBIUS_API_KEY"] = userdata.get("nebius_api_key")

from collections import defaultdict, deque
from openai import OpenAI
from typing import Dict, Any, Optional
import datetime
import string
import random
from dataclasses import dataclass

@dataclass
class NPCConfig:
    world_description: str
    character_description: str
    history_size: int = 10
    has_scratchpad: bool = False

class NPCFactoryError(Exception):
    """Base exception class for NPC Factory errors."""
    pass

class NPCNotFoundError(NPCFactoryError):
    """Raised when trying to interact with a non-existent NPC."""
    def __init__(self, npc_id: str):
        self.npc_id = npc_id
        super().__init__(f"NPC with ID '{npc_id}' not found")

class SimpleChatNPC:
    def __init__(self, client: OpenAI, model: str, config: NPCConfig):
        self.client = client
        self.model = model
        self.config = config
        self.chat_histories = defaultdict(lambda: deque(maxlen=config.history_size))

    def localize_input(self, input_text: str) -> str:
        """Translate user input into English."""
        return self.client.chat.completions.create(
            messages=[
                {
                    'role': 'user',
                    'content': f'Translate the following text into English:\n{input_text}. Only output the translation itself.'
                }
            ],
            model=self.model,
        ).choices[0].message.content

    def localize_output(self, output_text: str, target_language: str) -> str:
        """Translate the output into the target language."""
        return self.client.chat.completions.create(
            messages=[
                {
                    'role': 'user',
                    'content': f'Translate the following text into {target_language}:\n{output_text}. Only output the translation itself.'
                }
            ],
            model=self.model,
        ).choices[0].message.content

    def get_system_message(self) -> Dict[str, str]:
        """Returns the system message that defines the NPC's behavior."""
        character_description = self.config.character_description

        if self.config.has_scratchpad:
            character_description += """
You can use scratchpad for thinking before you answer: whatever you output between #SCRATCHPAD and #ANSWER won't be shown to anyone.
You start your output with #SCRATCHPAD and after you've done thinking, you #ANSWER"""

        return {
            "role": "system",
            "content": f"""WORLD SETTING: {self.config.world_description}
###
{character_description}"""
        }

    def chat(self, user_message: str, user_id: str, user_language: str) -> str:
        """Process a user message and return the NPC's response."""
        messages = [self.get_system_message()]

        # Add conversation history
        history = list(self.chat_histories[user_id])
        if history:
            messages.extend(history)

        # Add new user message
        if user_language != "English":
            user_message_dict = {
                "role": "user",
                "content": self.localize_input(user_message)
            }
        else:
            user_message_dict = {
                "role": "user",
                "content": user_message
            }
        self.chat_histories[user_id].append(user_message_dict)
        messages.append(user_message_dict)

        try:
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=0.6
            )

            response = completion.choices[0].message.content

            # Handle scratchpad if enabled
            response_clean = response
            if self.config.has_scratchpad:
                import re
                scratchpad_match = re.search(r"#SCRATCHPAD(:?)(.*?)#ANSWER(:?)", response, re.DOTALL)
                if scratchpad_match:
                    response_clean = response[scratchpad_match.end():].strip()

            if user_language != "English":
                response_clean = self.localize_output(response_clean, user_language)

            # Store response in history, including the scratchpad
            self.chat_histories[user_id].append({
                "role": "assistant",
                "content": response
            })

            # Return the message to the user without a scratchpad
            return response_clean

        except Exception as e:
            return f"Error: {str(e)}"

class NPCFactory:
    def __init__(self, client: OpenAI, model: str):
        self.client = client
        self.model = model
        self.npcs: Dict[str, SimpleChatNPC] = {}
        self.user_ids: Dict[str, str] = {}  # username -> user_id mapping
        self.user_preffered_languages: Dict[str, str] = {}  # user_id -> language mapping

    def set_user_language(self, user_id: str, language: str):
        """Set the preferred language for a user."""
        self.user_preffered_languages[user_id] = language

    def generate_id(self) -> str:
        """Generate a random unique identifier."""
        return ''.join(random.choice(string.ascii_letters) for _ in range(8))

    def register_user(self, username: str) -> str:
        """Register a new user and return their unique ID.
        If username already exists, appends a numerical suffix."""
        base_username = username
        suffix = 1

        # Keep trying with incremented suffixes until we find an unused name
        while username in self.user_ids:
            username = f"{base_username}_{suffix}"
            suffix += 1

        user_id = self.generate_id()
        self.user_ids[username] = user_id
        return user_id

    def register_npc(self, world_description: str, character_description: str,
                     history_size: int = 10, has_scratchpad: bool = False) -> str:
        """Create and register a new NPC, returning its unique ID."""
        npc_id = self.generate_id()

        config = NPCConfig(
            world_description=world_description,
            character_description=character_description,
            history_size=history_size,
            has_scratchpad=has_scratchpad
        )

        self.npcs[npc_id] = SimpleChatNPC(self.client, self.model, config)
        return npc_id

    def chat_with_npc(self, npc_id: str, user_id: str, message: str) -> str:
        """Send a message to a specific NPC from a specific user.

        Args:
            npc_id: The unique identifier of the NPC
            user_id: The unique identifier of the user
            message: The message to send

        Returns:
            The NPC's response

        Raises:
            NPCNotFoundError: If the specified NPC doesn't exist
        """
        user_language = self.user_preffered_languages.get(user_id, "English")
        if npc_id not in self.npcs:
            raise NPCNotFoundError(npc_id)

        npc = self.npcs[npc_id]
        return npc.chat(message, user_id, user_language=self.user_preffered_languages[user_id])

    def get_npc_chat_history(self, npc_id: str, user_id: str) -> list:
        """Retrieve chat history between a specific user and NPC.

        Args:
            npc_id: The unique identifier of the NPC
            user_id: The unique identifier of the user

        Returns:
            List of message dictionaries containing the chat history

        Raises:
            NPCNotFoundError: If the specified NPC doesn't exist
        """
        if npc_id not in self.npcs:
            raise NPCNotFoundError(npc_id)

        return list(self.npcs[npc_id].chat_histories[user_id])

In [45]:
from openai import OpenAI

# Nebius uses the same OpenAI() class, but with additional details
client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

model = "meta-llama/Meta-Llama-3.1-405B-Instruct"

# Creating a factory
npc_factory = NPCFactory(client=client, model=model)

In [47]:
# Register a user
user_id = npc_factory.register_user("Alice")

# Set user preffered language
preffered_language = "Old English"
npc_factory.set_user_language(user_id, preffered_language)

# Create an NPC
npc_id = npc_factory.register_npc(
    world_description="Medieval London, XIII century",
    character_description="A knight at Edward I's court",
    has_scratchpad=False
)

In [48]:
def prettify_string(text, max_line_length=80):
    """Prints a string with line breaks at spaces to prevent horizontal scrolling.

    Args:
        text: The string to print.
        max_line_length: The maximum length of each line.
    """

    output_lines = []
    lines = text.split("\n")
    for line in lines:
        current_line = ""
        words = line.split()
        for word in words:
            if len(current_line) + len(word) + 1 <= max_line_length:
                current_line += word + " "
            else:
                output_lines.append(current_line.strip())
                current_line = word + " "
        output_lines.append(current_line.strip())  # Append the last line
    return "\n".join(output_lines)

In [50]:
# We can hack our own code a bit and use translation features to test the system.

npc = npc_factory.npcs[npc_id]

message = "Hello, who are you and what brings you here?"
message_translated = npc.localize_output(message, preffered_language)
print(f"Translated message: {message_translated}")

response = npc_factory.chat_with_npc(
    npc_id,
    user_id,
    message_translated
)
print("Original answer")
print(prettify_string(response))
print("Translated answer")
print(prettify_string(npc.localize_input(response)))

Translated message: Hailo, hwā eart þu and hwæt bringeð þe hider?
Original answer
Gōd morgen þē. Ic eom Sir Eadweard de Muntfort, ān eadmod cniht on þēowdōme
ūres mīclan Cyning Eadweard I. 'Tis wyrþ tō wician hēafod and þēowian þone
crīadan. Ic hæbbe cōme tō Lundenbyrig tō bidden mīne arwyrþnesse þon cyning and
tō tēonan on þone heah getil on his wyrþnesse.

Sōþlīce, ic mūst andettan þæt ic eom hēr swylce tō sēcan giefþe mid þon cyning,
þæt ic mæg beadon þone dōm mīnes hlǣford, þes Eorl of Lēgreceastre, sē þe hæbbe
unrihtlīce benamed geboren fram sylfa hlēohan æt hēafod. Ic hōpe tō clǣnan his
gōde naman and tō steren his wyrþ on þæs cyninges ēagan.

Gebēad þē, sæg mē, hwā eart þū? Eart þū hlæford, cēpingmanna, oþþe hwilc sǣ
oþer lique cleric? Hwæt bringþ þē tō þyss fæstan byrig?
Translated answer
Good morning to you. I am Sir Edward de Montfort, a humble knight in the
service of our great King Edward I. It is worthy to bow your head and serve the
crown. I have come to London to beg for

## Task 2. Translating poetry

LLMs are notoriously bad at translating poetry. The resulting poems rarely have good rhyme and rhytm. Let's try to naively translate some Humpty Dumpty rhyme to a language of your choice:

In [51]:
language = "Dutch"
poem = """Humpty Dumpty sat on a wall,
Humpty Dumpty had a great fall.
All the king's horses and all the king's men
Couldn't put Humpty together again."""

print(answer_with_llm(
    f"Translate the following children's rhyme to {language}\n{poem}"
))

Here's the translation of the children's rhyme to Dutch:

Humpty Dumpty zat op een muur,
Humpty Dumpty maakte een grote val.
Alle paarden van de koning en alle mannen van de koning
Konden Humpty niet meer in elkaar zetten.

Note: The name "Humpty Dumpty" is often left untranslated in Dutch, as it's a
well-known character. If you want to translate the name as well, you could use
"Lammy Dommy" or "Eieltje" (which means "little egg"), but "Humpty Dumpty" is
more commonly used in Dutch.


This is hardly a good translation because the rhyme is completely broken.

You task is to create a chain of LLM calls to do the following steps in translating a poem:

1. Do a naive literal translation to preserve the meaning
2. Rewrite the translation to retain rhyme and rhythm but perhaps loosing a bit of meaning.
3. Finally have an editor look at both original and translation and make final touch ups.

We encourage you to try and prompt your LLMs to do the job of "Translator", "Editor" and so on.
Also choose the language you can understand best so that you can evaluate the result well (you can also change the original to some other language)

In [53]:
literal_translator_system_prompt = f"""
You are a translator, whose task is to translate whatever you receive literally to {language}.
You don't care if the text changes sentences, length, anything else, you only task is to retain as much meaning as possible
and be as literal as possible in your translation.
Output 'Translation:' and then the translation you created.
"""

rhyme_rewriter_system_prompt = f"""
You are a rhyme writer. You task is to receive a text in {language} and rewrite
it in a rhymed way also in {language}. It should have rhymes following one of the popular patterns,
for example every other line, two and two and so on.
You can distort the meaning a bit but not to lose it completely.
Try to not make the text much longer.
Output 'Rewriting:' and then the rewriting you created.
"""

editor_system_prompt = f"""
You are an editor.
You task is to inspect the the original text, naive translation and naive rewriting.
Assess the quality of all of them and make finishing touch ups on the rhymed rewriting.
Do not the text much longer than the original, it should have the same amount of lines.
Do not output anything after the translation.
Output the remarks you have for it in English and then
Output 'Final translation:' and the final version of the translation to {language}
"""


def translate_in_stages(input: str, language: str) -> str:
    naive_translation = answer_with_llm(
        system_prompt=literal_translator_system_prompt,
        prompt=input
    )
    print(f"\n\n{naive_translation}\n\n")

    rhymed_text = answer_with_llm(
        system_prompt=rhyme_rewriter_system_prompt,
        prompt=naive_translation
    )
    print(f"\n\n{rhymed_text}\n\n")

    editor_notes = answer_with_llm(
        system_prompt=editor_system_prompt,
        prompt=f"""
Original_text: {input}
Naive translation: {naive_translation}
Rhymed rewriting: {rhymed_text}
"""
    )
    print(f"\n\n{editor_notes}\n\n")

    return editor_notes.split("Final translation:")[1]

In [54]:
translate_in_stages(poem, language)



Translation:
Humpty Dumpty zat op een muur,
Humpty Dumpty had een grote val.
Alle paarden van de koning en alle mannen van de koning
Konden Humpty niet weer samen zetten.




Rewriting:
Humpty Dumpty zat op een hoge muur zo steil,
En maakte een val, wat een groteuil.
De koning stuurde zijn paarden met spoed,
En zijn mannen ook, maar Humpty bleef gebroken in de nood.
Ze probeerden en probeerden, maar tevergeefs, helaas,
Humpty Dumpty bleef stuk, en dat was zijn grote val, dat is waar.




The original text is a well-known English nursery rhyme with a consistent rhyme
scheme and meter.
The naive translation is a literal translation of the original text, but it
doesn't quite capture the same rhythm and rhyme scheme.
The rhymed rewriting attempts to improve upon the naive translation by using a
more natural-sounding Dutch phrase structure and incorporating rhymes, but it
has some issues with meter and line length, and also adds an extra line that
disrupts the original's concise structure

'\nHumpty Dumpty zat op een muur zo hoog,\nHumpty Dumpty had een grote val, een groot verdrietig lied.\nAlle paarden van de koning en alle mannen van de koning\nKonden Humpty niet weer samen zetten, hij bleef gebroken, dat is waar.'

## Task 3. Finding a hero

Our kingdom has a very formal process for approving a hero for a specific quest.
You task is to implement the approvement process using LLM calls:

You receive a request for a hero and a description of a hero to hire for this quest.

**Step 1**. Check that request is formally correct. You can come up with your own ideas, but we suggest the following criteria:
- It has a name of the person requesting a hero and a date;
- It has a description of who's going to supply the hero with money and other resources;
- It has a reason why the hero is needed, some quest or challenge;
- It has a recommended qualification for the hero;

**Step 2**. Check that the problem with which the request is trying to deal is sufficient to actually find a hero, or perhaps an author might do it themself or find an easier solution.

**Step 3**. Make sure that the description of the hero is compatible with the quest and requirements placed on the hero.

These steps should be performed sequentially, one after another. If any of the stages fail, immediately return a refusal with justification - you don't want to waste any more compute on unworthy queries! If all the three steps succeed, return "accepted".

In [55]:
formal_checker_system_prompt = """
You are presented with a request to hire a hero, check the following formalities:
It has a name of the person requesting a hero and a date;
It has a description of who's going to supply the hero with money and resource;
It has a reason why the hero is needed, quest they are going to complete;
It has a recommended qualifications for the hero;
If any of those isn't true output the following:
REFUSE: (here some brief justification why)
If all of the criteria are good, output 'ACCEPT'
"""

problem_scale_checker = """
You are presented with a request to hire a hero.
Access whether the problem described in the request is worthy of looking for a hero to solve.
For example:
Slaying a dragon - good
Buying apples - bad
Playing trumpet - bad
Delivering sacred artifact - good.
In general the problem should be quiet epic to be good.
If it's not good output:
REFUSE: (here briefly justify why itn't not fitting)
If it's good output:
'ACCEPT'
"""


consistency_checker_prompt = """
You are presented with a request to hire a hero and the hero's description.
You should make sure that whatever the problem described in the request
can actually be solved by the hero proposed, based on the requirements in
the request and the qualities of the hero presented.
If the hero is fitting, output: 'ACCEPT'
Otherwise output:
REFUSE: (some brief justification why we reject the hero for this request)
"""


def check_hero_request(request_for_a_hero: str, hero_descripition: str) -> str:
    formal_check = answer_with_llm(
        system_prompt=formal_checker_system_prompt,
        prompt=request_for_a_hero
    )
    if "REFUSE" in formal_check:
        return False, formal_check


    problem_scale_check = answer_with_llm(
        system_prompt=problem_scale_checker,
        prompt=request_for_a_hero
    )
    if "REFUSE" in problem_scale_check:
        return False, problem_scale_check

    consistency_check = answer_with_llm(
        system_prompt=consistency_checker_prompt,
        prompt=f"Request: {request_for_a_hero}, hero_description: {hero_descripition}"
    )
    if "REFUSE" in consistency_check:
        return False, consistency_check

    return True, ''


In [56]:
epic_request = """
Epic Hero Request
Name of Requestor: Lord Aeldric of the Silver Vale
Date: The 15th Day of Bloomrise, Year 1025 of the Dawnstar Calendar

Resource Provision:
The hero shall be provisioned by the Guild of Eternal Flame, a conclave of wealthy artificers and arcane financiers, who have pledged a sum of 500,000 gold crowns, enchanted arms and armor, rare tomes of forgotten magic, a sky-bound griffon steed, and a personal aide skilled in healing and reconnaissance. All resources shall be delivered at the Hall of Summoning in Ironhold.

Purpose of Request / Quest Description:
Darkness stirs in the Hollow Spine Mountains, where the Obsidian Serpent — an ancient beast thought long dead — has risen anew. Villages lie in ruin, and the skies turn black with ash. The hero is summoned to descend into the Abyssal Breach, recover the lost Emberheart Crystal, and seal the rift before the World Spine fractures and all realms fall into chaos.

Recommended Hero Qualifications:

Proven mastery in combat, both arcane and martial

Experience in surviving extreme environments and demonic incursions

Wisdom enough to resist corruption, and strength to slay without hesitation

Familiarity with ancient dialects and lost technologies

A heart unwavering in the face of despair, and a spirit unbreakable by shadow

Let the stars guide the right soul to answer. The fate of the realms balances on a blade’s edge.
"""

not_epic_request = """
Epic Hero Request
Name of Requestor: Steve
Date: 3rd of January 2025

Resource Provision:
I'll pay from my pocket

Quest Description:
I need someone to run to a supermarket for me, i'm hungry

Required qualification:
- Be very fast
- Be smart to buy good snacks.
"""

wrong_request = """
Hello, I need a mighty warrior to slay evil, thank you!
"""

In [57]:
epic_hero = """
Hero Profile: Kaelen Thorne, the Ash-Wrought Sentinel

Forged in the fires of the Blistering Wars and tempered by years wandering the haunted ruins of the Old Kingdoms, Kaelen Thorne is a battle-scarred veteran clad in rune-etched obsidian armor. With one eye gifted by the Seers of Valemire—able to glimpse the truth behind illusions—and a blade forged from a fallen star, Kaelen walks the line between light and shadow.

Equal parts scholar and warrior, Kaelen speaks the tongues of forgotten realms and wields spells that twist the very air. Haunted but unyielding, Kaelen has turned away crowns and glory before—but for a quest that may decide the fate of all creation, the Sentinel rises once more.
"""

not_so_epic_hero = """
Tom the cat

Is a cat, supposed to catch mice, but can't really do it.
Has a lot of different surprising weapons and contraptions, but they always work against them.

Works for cat food.
"""

In [58]:
check_hero_request(request_for_a_hero=epic_request, hero_descripition=epic_hero)

(True, '')

In [59]:
check_hero_request(request_for_a_hero=epic_request, hero_descripition=not_so_epic_hero)

(False,
 "REFUSE: The hero, Tom the cat, does not meet the recommended qualifications for\nthe quest. Tom's inability to catch mice, a relatively simple task, raises\nconcerns about his combat mastery and effectiveness in surviving extreme\nenvironments. Additionally, there is no indication that Tom possesses the\nnecessary wisdom, strength, or familiarity with ancient dialects and lost\ntechnologies to complete the task. The fate of the realms requires a more\ncapable and reliable hero.")

In [60]:
check_hero_request(request_for_a_hero=not_epic_request, hero_descripition=epic_hero)

(False,
 "REFUSE: The task of running to a supermarket to buy snacks is a mundane,\neveryday chore that does not require heroic intervention. It lacks the epic\nscale and significance typically associated with quests worthy of a hero's\nattention.")

In [61]:
check_hero_request(request_for_a_hero=wrong_request, hero_descripition=epic_hero)

(False,
 'REFUSE: The request lacks essential formalities, including the name of the\nperson requesting a hero, a date, a description of who will supply the hero\nwith money and resources, and recommended qualifications for the hero.')